<a href="https://colab.research.google.com/github/jameramellyn/skincare_project/blob/not-k-cross-validation/Copy_of_SkinTypeDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
!pip install Pillow

from numpy import asarray

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
images = '/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types'

/content


In [ ]:
#the dataset only comes with images that have been sorted into folders, but not formally labelled
#this function will help to label each of the images as the dataframe is being created
label_index = {"dry": 0, "normal": 1, "oily": 2}
index_label = {0: "dry", 1: "normal", 2: "oily"}

def create_df(base):
    dd = {"images": [], "labels": []}
    for i in os.listdir(base):
        label = os.path.join(base, i)
        for j in os.listdir(label):
            img = os.path.join(label, j)
            dd["images"] += [img]
            dd["labels"] += [label_index[i]]
    return pd.DataFrame(dd)

train_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/train")
val_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/valid")
test_df = create_df("/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/test")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SkincareProject/Oily-Dry-Skin-Types/train'

In [ ]:
def load_image(file_path):
    with Image.open(file_path) as img:
        img = img.convert('RGB')  # Ensure image is in RGB mode
        return np.array(img)

def extract_image_data(file_path):
    img_array = load_image(file_path)
    height, width, channels = img_array.shape
    return pd.Series({
        'height': height,
        'width': width,
        'channels': channels,
        'image_array': img_array
    })

# Apply the function to each file path in the dataframe
train_df_data = train_df.iloc[:, 0].apply(extract_image_data)
val_df_data = val_df.iloc[:, 0].apply(extract_image_data)
test_df_data = test_df.iloc[:, 0].apply(extract_image_data)

# Join the results with the original dataframe
train_df = train_df.join(train_df_data)
val_df = val_df.join(val_df_data)
test_df = test_df.join(test_df_data)



In [ ]:
train_df_array = asarray(train_df)
# Add a new dimension at the beginning (axis=0)
train_df_new = np.expand_dims(train_df_array, axis=0)
train_df_new = train_df_new[:,:,:3]

print(type(train_df_new))
print(train_df_new)
train_df_new.shape

In [ ]:
# Step 1: Extract the image arrays from the DataFrame
x_test = np.stack(train_df['image_array'].values)

# Step 2: Ensure the shape is correct
# x_test should already have the shape (num_images, height, width, channels)
# but you can verify or reshape if necessary:
x_test = x_test.reshape((len(x_test), 640, 640, 3))

# Step 3: Normalize the images
x_test = x_test / 255.0

In [ ]:
#x is images, y is labels
x_train = train_df['images']
y_train = train_df['labels']
x_test = test_df['images']
y_test = test_df['labels']

# normalize
def normalize(df):
  j =0
  for i in df:
    im = Image.open(i[0])
    pixels = asarray(im)
    pixels = pixels.astype('float32')
    pixels /= 255.0
    j =j+ 1
    print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))
    print(j)


normalize(x_train)
normalize(x_test)

In [ ]:
# Reshape the images.
def expand(df):


x_train = np.expand_dims(x_train, axis=1)
x_test = np.expand_dims(x_test, axis=1)

In [ ]:


#visualizing images

def plot_example_images(df, num_classes=3, num_images_per_class=5):
    fig, axes = plt.subplots(num_classes, num_images_per_class, figsize=(15, 5))

    for class_id in range(num_classes):
        class_images = df[df['labels'] == class_id].sample(num_images_per_class)
        for i, (index, row) in enumerate(class_images.iterrows()):
            img = Image.open(row['images'])
            axes[class_id, i].imshow(img)
            axes[class_id, i].axis('off')
            if i == 0:
                axes[class_id, i].set_title(f'Class {class_id}')

    plt.tight_layout()
    plt.show()

plot_example_images(train_df)

In [ ]:
# create CNN structure
model = Sequential()
model.add(Input(shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) # notice no MaxPool after this one
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

# compile and train model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_oh, epochs=10, batch_size=32, validation_data=(x_val, y_val_oh))

# evaluate accuracy
y_pred = model.predict(x_test)
y_pred = tf.argmax(y_pred, axis=1)
# y_test = tf.argmax(y_test_oh, axis=1)
loss, acc = model.evaluate(x_test, y_test_oh)
print("Test accuracy:", acc)
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

